In [16]:
import pandas as pd
import pprint
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from nltk.corpus import stopwords 
import nltk
import csv
import numpy as np
import re
import emoji
from keras.models import Sequential
from keras.layers import Dense, Dropout
from nltk.stem.porter import PorterStemmer
from string import digits
from autocorrect import spell
import preprocessor as p
import requests
import json
import wordsegment as ws
from nltk.stem.wordnet import WordNetLemmatizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
stemmer = PorterStemmer()
ws.load()
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
from nltk.corpus.reader.wordnet import NOUN

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dileepjayamal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dileepjayamal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Pre processing - PoS tagging


In [2]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return NOUN # for easy if-statement 


In [3]:

def format_tweet(row):
    
    p.set_options(p.OPT.HASHTAG, p.OPT.NUMBER, p.OPT.EMOJI, p.OPT.SMILEY,p.OPT.URL, p.OPT.MENTION )
    parsed_tweet = p.parse(row)
    extra = []
    if parsed_tweet.urls:
        for each in parsed_tweet.urls:
            row = row[:each.start_index]+ ' URL '+row[each.end_index:]
            
    if parsed_tweet.mentions:
        for each in parsed_tweet.mentions:
            row = row[:each.start_index]+ ' Smith '+row[each.end_index:]
            
    if parsed_tweet.hashtags:
        for each in parsed_tweet.hashtags:
            extra+=ws.segment(each.match)
    
    
    p.set_options(p.OPT.URL, p.OPT.MENTION , p.OPT.NUMBER)
    row = p.clean(row)
    tokens = tknzr.tokenize(row)
    tokens_formatted = []
    #count=0
    #for i in range(len(tokens)):
    #    if tokens[i] in emoji.UNICODE_EMOJI:
    #        tokens[i] = emoji.demojize(tokens[i])
            
    for each in tokens:
        if len(each)> 0:
            tokens_formatted.append(each)
            
    tokens_formatted = nltk.pos_tag(tokens_formatted)
    tags = []
    for i in range(len(tokens_formatted)):
        
        tags.append(tokens_formatted[i][1])
        tokens_formatted[i]=lemmatizer.lemmatize(tokens_formatted[i][0],pos = get_wordnet_pos(tokens_formatted[i][1]))
    tokens_formatted+=extra
    return " ".join(tokens_formatted+tags).lower()

In [4]:
def format_data(row_data):
    data = []
    i=0
    for row in row_data:
        data.append(format_tweet(row))
    return data

In [5]:
data = []
classes = []

with open('../data/text_formatted.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        try:
            classes.append(int(row[1]))
        except:
            continue
        data.append(row[2])
traing_data = format_data(data)
i=0
for cl,d in zip(classes,traing_data):
    i+=1
    print(i," ",cl," : ",d)

1   1  :  sweet united nations video . just in time for christmas . #imagine #noreligion #irony url imagine no religion irony nnp nnp nnp nn . rb in nn in nnp . vb nn nnp nnp
2   1  :  smith we be rumor to have talk to erv's agent ... and the angels ask about ed escobar ... that's hardly nothing #sarcasm ;) sarcasm nnp prp vbp vbn to vb vbn to nnp nn : cc dt nnps vbd in nnp nnp : nn rb nn nn nn
3   1  :  hey there ! nice to see you minnesota / nd winter weather #not not nnp ex . nnp to vb prp nnp nnp nnp nnp nnp nn
4   0  :  episode leave i'm die over here nns vbd nnp vbg in rb
5   1  :  i can't breathe ! be choose a the most notable quote of the year in an annual list release by a yale university librarian #irony irony prp vbp rb . vbd vbn in dt rbs jj nn in dt nn in dt jj nn vbn in dt nnp nnp jj nn
6   0  :  you're never too old for footie pajamas . url rb rb rb jj in nnp nnp . nn
7   1  :  nothing make me happy then get on the highway and see break light light up like a christmas tr

392   0  :  dats how u kno it winter url vb wrb jj vb prp$ nn nn
393   1  :  i like creepy guy that ask me random question at swic #not not prp vbp jj nns wdt vbp prp jj nns in nn nn
394   1  :  someone stop obama ! usa only should have relation with human right love nation like saudi arabia . not terrorist cuba ! #cuba #sarcasm cuba sarcasm nn vbd nnp . nnp rb md vb nns in jj nns vbg nns in nnp nnp . rb jj nnp . nn nn
395   0  :  watch father and not his daughter fuck url #old + young #father #and #not old father and not nnp nn cc rb prp$ nn nnp nnp nnp nnp nnp nnp nnp nn
396   1  :  " smith no nn nnp nnp
397   1  :  marvin lewis clearly think very highly of johnny football #not #norespect smith not no respect nnp nnp rb vbz rb rb in nnp nnp nnp nnp nnp
398   1  :  finals week be drain my body so much that it's ruin my life in other place . lovely ... #not not nns nnp vbz vbg prp$ nn rb jj in nn vbg prp$ nn in jj nns . rb : nn
399   1  :  about to write my math final really excited #n

768   1  :  " my dove wrapper tell me to be mischievous vb nnp nn nn vbd prp to vb jj
769   0  :  smith * kiss you cheek * that's enough . im not a love bot . nnp nnp vbz prp vbp jj nn rb . vb rb dt nn nn .
770   1  :  smith smith be he band from wear the clothes ? #irony #karma irony karma nnp nnp vbz prp vbd in vbg dt nns . nn nn
771   1  :  smith yeah that's it #sarcasm sarcasm nnp prp vbd prp vb
772   1  :  " hula hoop jj nn nns
773   0  :  they be still try to decide if it's terrorism ? ? omg ! #sydneysiege sydney siege prp vbp rb vbg to vb in jj nn . . uh . nn
774   1  :  the fandom show it mature side a always when it's about this kind of topic #irony irony dt nn vbg prp$ jj nn in rb wrb nn in dt nn in nn nn
775   1  :  i love put flobeam mechs back together . | | #not not prp vbp vbg nnp nn rb rb . vb cd nn
776   0  :  like tyndall's system i just think robert hubbs should play more he give u another scorer and a consistent pt shooter that our o need in nnp nn prp rb vbp nnp nn

1092   0  :  there be two thing that need to well . drafting and the player work ethic . that's it . ex vbp cd nns wdt vbp to rbr . vbg cc dt nns vbp jj . vb prp .
1093   0  :  " when someone on a conference call talk about " " housekeep item " " . grrr . " nn wrb nn in dt nn nn nns in nnp nnp vbg nns jj nnp . nnp . nn
1094   1  :  youre such a great person . #not not nn pdt dt jj nn . nn
1095   1  :  saturday class wee #sarcasm sarcasm jj nn nn nn
1096   1  :  smith i don't recall dalai lama talk about busty girl and celebs but apart from that ... #irony irony nnp prp vbp vb jj nn vbg in jj nns cc nns cc rb in dt : nn
1097   1  :  " went off the road this morning a do my english professor nns vbd in dt nn dt nn in vbd prp$ nn nn
1098   0  :  smith how many championship ? you try to say that make him worth more than saban ? nnp wrb jj nns . prp vbg to vb in vbz prp in jjr in nnp .
1099   0  :  smith should just relate to her water first amen nnp md rb vb to prp$ nn jj nnp
1100   1  :  

1492   1  :  i just love when you test my patience ! ☺ ️ #not not prp rb vb wrb prp vbp prp$ nn . jj cd nn
1493   1  :  welcome to twitter smith myself n smith r at ur service a ur twitter coach #sarcasm sarcasm vb to vb nnp prp rb nnp nn in jj nn in jj nn nns vbp
1494   1  :  smith #thingsbetterthantitansjags napalm hotwings #bigjoefavorite #sarcasm love the show #keep it real things better than titans jags big joe favorite sarcasm keep nnp nnp jj nns vbp jj in dt nn vbz prp jj
1495   0  :  sometimes i can make funny comment #boyfriend #text #justgothomefromdinner #smiles #funny #sarcasm #lovehim url boyfriend text just got home from dinner smiles funny sarcasm love him rb prp md vb jj nns vbp jj nn nns vbp nnp nnp nnp
1496   0  :  " dayum nn nnp
1497   0  :  #not a bad idea ! url not vb dt jj nn . nn
1498   1  :  you know when you send present to friend who live far away and they never get there .. and you handmade most of the stuff .. yay #not not prp vbp wrb prp vbp nns to nns wp v

1891   1  :  #socialmedia where it's perfectly normal to talk to people in picture ! #sarcasm social media sarcasm nn wrb rb rb jj to vb to nns in nns . nn
1892   1  :  " smith hey nn nnp nnp
1893   1  :  " smith re : xmas : " " christmas " " be itself an abbreviation of " " christ's mass . " " #irony smith irony nn nnp nnp : nn : jj nnp nnp nnp nnp vbz prp dt nn in nnp nnp nnp nnp . vb jj nn nnp
1894   1  :  smith now that's just mean . #sarcasm #sorrynotsorry sarcasm sorry not sorry nnp rb vbz rb vb . nn nn
1895   1  :  how do we end up talk ? .. oh wait .. we don't talk .. whooops #cutestcouple #not cutest couple not wrb vbp prp vb rp vbg . jj jj nn nn prp vbp vb nnp nns rb vbp
1896   0  :  a year ago this would be just a writing on a t-shirt #not anymore all thanks to smith smith ryxmas url not merry xmas dt nn in dt md vb rb dt nn in dt jj nn rb dt nns to nnp nnp nn nnp
1897   0  :  goodie bag for nd grade be do ! #whatsnext ? whats next nnp nns in jj nn vbp vbn . nn .
1898   1  :

2291   1  :  smith woww thanks for that help alot #not hahaha saddd 😂 not nnp jj nns in dt vbz rb jj nn nn nn
2292   1  :  smith it's not work if you enjoy it #eh #not eh not nnp nnp rb vb in prp vbp prp vbz nn
2293   0  :  when all else fail ... url #relationships #needs via smith relationships needs wrb dt rb vbz : nnp nnp vbz in nnp
2294   0  :  gonna be the shittest christmas ever without my mum .. gonna get so drunk 🍹 🍹 🍹 nnp vb dt jjs nnps rb in prp$ jj nnp nnp vb rb jj nn nnp nn
2295   0  :  " smith im sorry chode nn nnp nnp nn nn
2296   1  :  another smith #irony moment . we can't be bother to fix your broadband problem but would you like to buy an iphone 6 of u ! hell no ! ! irony dt nnp nnp nn . prp vbp vb vbn to vb prp$ nn nn cc md prp in to vb dt nn cd in prp . nnp uh . .
2297   1  :  " smith smith steveau smith u have more ambition john nn nnp nnp nn nnp nn vbp jjr nn nn
2298   1  :  " now that i can seem to afford good thing rb rb in nn md vb to vb jj nns
2299   1  :  you

2691   0  :  " smith we all love a trier mate jj nnp prp dt vbp dt nn nn
2692   0  :  the mouse's first incepted memory be just the sound : braaawwwp ! dt nn rb vbd nn vbd rb dt nn : nn .
2693   0  :  #not a bad gift : wall gives gooden $ 100k cash ( via url ) url not vb dt nnp nnp : nnp vbz nnp $ cd nnp ( in nnp ) nn
2694   0  :  porygon 2 be #not find in the url . com wild . #firemen follow smith not firemen paint nn cd vbp vbn vbn in dt nnp . nn jj . nns vbp nnp
2695   0  :  " #opec #chief #defends #policy opec chief defends policy jj nnp nnp vbz nn
2696   0  :  smith i think i'm gonna check your music out later . thanks ☺ nnp prp vbp nnp fw vb prp$ nn rp rb . nns vbp
2697   1  :  " smith careful nn nnp nnp
2698   1  :  you're go and i gotta stay high ... #coldpants #offtomysteriousplaces #not #bye url cold pants off to mysterious places not bye nnp vbn cc prp vbp jj jj : nns vbz jj nnp nnp
2699   1  :  smith smith remember when win one regular season game be like a super bowl #not 

3091   1  :  smith thats not weird at all #sarcasm sarcasm nnp nns rb vb in dt nn
3092   0  :  i work in front of laptop for hour a day and suffer terrible dry eye . #bloodshot #irony #eyerony bloodshot irony eye rony prp vbp in nn in nn in nns dt nn cc nn jj jj nn . jj nn nn
3093   0  :  bummer end ... but what a fun game and season . #gophers gophers nnp vbg : cc wp dt nn nn cc nn . nns
3094   0  :  dallas ( - 7 ) over washington ; new orleans ( - 7 ) over tampa bay ; green bay over detroit ( + 9 ); carolina ( +2.5 ) over atlanta . nnp ( : cd ) in nnp : nnp nnp ( : cd ) in nnp nnp : nnp nnp in nnp ( jj cd jj nnp ( nnp ) in nnp .
3095   0  :  smith smith r smith aymu smith ycfc smith eva fu smith i smith rch eastenders #not not nnp nnp nn nnp nn nnp nn nnp nn nn nnp nn nnp nn nns vbp
3096   1  :  i didn't know the story of noah have transformer in it . man the modern biblical tale kill it . #not #whatarethelivingrockpeoplefor not what are the living rock people for prp vbp vb dt nn in

3491   1  :  that - hour sleep put me into hyper mode #irony irony dt : nn nn vbd prp in jj nn nn
3492   1  :  twirra be not a place . rt smith : twitter a place where virgin will be tweet sexual & hoe will be forming saint ... #irony irony nnp vbz nnp dt nn . nnp nnp : nn dt nn wrb nns md vb vbg jj cc nns md vb jj nn : nn
3493   1  :  the grinch stole my sanity url via #lol #fun # smith #sarcasm url lol fun funny sarcasm dt nnp nnp nnp nnp nnp in nnp nnp # nnp nnp nnp
3494   1  :  so despite the fact that i be up till study and have to wake up at i think this day be go to be great 😒 #sarcasm sarcasm rb in dt nn in prp vbd rb jj vbg cc vbd to vb rp in prp vbp dt nn vbz vbg to vb jj jj nn
3495   1  :  i be really happy to hear that all of the nbc hockey talk head point out ovechkin's strong corsi & + 4 status this season . #sarcasm sarcasm prp vbd rb jj to vb in dt in dt nnp nn vbg nns vbd rp nnp jj nnp cc nnp cd nn dt nn . nn
3496   0  :  #usa : senate summary report on #cia detention 

In [6]:
submission_test_set_row = []
with open('../data/test_formatted.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        try:
            row = row[1]
        except Exceptions as ex:
            print(ex)
            continue
        submission_test_set_row.append(row)
submission_test_set_formatted = format_data(submission_test_set_row)
submission_test_set_formatted

['smith can u help ? | | more conservative need on #tsu + get pay post stuff like this ! | | you $ can go to url tsu nnp nnp nnp nnp . vb in jjr nns vbn in nnp nnp vb vbn vbg nn in dt . nn in nnp $ md vb to vb',
 '" just walk in to #starbucks and ask for a " " tall blonde " " hahahaha #irony " starbucks irony nn nnp vbd in to nns cc vbd in dt jj nnp nn nn nnp nnp nnp nnp nn',
 '#not gonna win url not nn nnp nnp nnp',
 'smith he be exactly that sort of person . weirdo ! nnp prp vbz rb in nn in nn . uh .',
 "so much #sarcasm at work mate / 10 #boring % #dead mate full on #shit absolutely #sleeping mate can't handle the #sarcasm sarcasm boring dead shit sleeping sarcasm rb jj nn in nn nn vbd cd vbg nn jj nn jj in nnp rb vbg jj nn vbd dt nn",
 'corny joke be my absolute favorite nnp nns vbp prp$ jj nn',
 '" people complain about my backround pic and all i feel be like " " hey don\'t blame me jj nns vbp in prp$ nn nn cc dt prp vbp vbz in nnp nnp vbp nn vb prp',
 '" smith smith rn nn nnp nnp

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1,5), min_df=2 )

X_train_counts = count_vect.fit_transform(traing_data+submission_test_set_formatted)
X_train_counts.shape


(4601, 28138)

In [9]:
from sklearn.feature_selection import SelectKBest, chi2,f_classif, f_regression, mutual_info_classif,mutual_info_regression
slcter = SelectKBest(chi2, k=256)
X_new = slcter.fit_transform(X_train_counts[:-784], classes)

X_new = slcter.transform(X_train_counts)
X_new.shape

(4601, 256)

In [10]:
traing_transformed = X_new[:-784]
print(traing_transformed.shape)


(3817, 256)


In [11]:
fraction = 3500
svd_transformed =traing_transformed
X_test = svd_transformed[fraction:]
Y_test = classes[fraction:]
X_train = svd_transformed[:fraction]
y_train = classes[:fraction]
#print(Y_test)

##  Train models

In [12]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

clf_sgd = SGDClassifier(alpha = 0.006,loss =  'log')
clf_sgd.fit(X_train, y_train)

print(clf_sgd.score(X_train, y_train))
clf_sgd.score(X_test,Y_test)


0.8148571428571428


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.7886435331230284

In [13]:
from sklearn.svm import SVC
clf_svc = SVC(C=0.09,gamma='auto',kernel="linear",random_state=10,probability=True)
clf_svc.fit(X_train, y_train)
print(clf_svc.score(X_train, y_train))
clf_svc.score(X_test,Y_test)


0.8391428571428572


0.8296529968454258

In [14]:
X_submission_counts = count_vect.transform(submission_test_set_formatted)
X_submission_counts_selected = slcter.transform(X_submission_counts)
print(X_submission_counts_selected.shape)
predicted_clf_svc = clf_svc.predict(X_submission_counts_selected)
predicted=predicted_clf_svc

(784, 256)


In [15]:
import csv
with open('../results/rs.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Index','Label'])
    for i in range(len(predicted)):
        writer.writerow([i+1,predicted[i]])